# hana-ml Tutorial - Dataframe

**Author: TI HDA DB HANA Core CN**

The SAP HANA Python Client API for machine learning algorithms (Python Client API for ML) provides a set of client-side Python functions for accessing and querying SAP HANA data, and a set of functions for developing machine learning models.
In this tutorial, we will show you how to use various functions of Dataframe. 

A dataframe represents a table (or any SQL statement).  Most operations on a dataframe are designed to not bring data back from the database unless explicitly asked for.

## Import necessary libraries and functions

In [ ]:
%run ./02-setup.ipynb

In [ ]:
from hana_ml import dataframe
from hana_ml.dataframe import ConnectionContext
from hana_ml.algorithms.pal.utility import DataSets, Settings
import pandas as pd

## Create a connection to a SAP HANA instance

First, you need to create a connetion to a SAP HANA instance. In the following cell, we use a config file, config/e2edata.ini to control the connection parameters. 

In your case, please update the following url, port, user, pwd with your HANA instance information for setting up the connection. 

In [ ]:
# Please replace url, port, user, pwd with your HANA instance information
connection_context = ConnectionContext(host_address, hdb_port, hdb_user, hdb_password)

## Obtain a hana-ml dataframe

**1. Create a hana-ml dataframe from pandas DataFrame**

create_dataframe_from_pandas() will create a dataframe from a pandas DataFrame and create a table in SAP HANA.

In [ ]:
df = dataframe.create_dataframe_from_pandas(connection_context=connection_context,
                                            pandas_df=pd.DataFrame({"ID": [1,2,5],
                                                                    "ID2": [1,None,5],
                                                                    "V3": [2,3,4],
                                                                    "V4": [3,3,3],
                                                                    "V5": ['a', None, 'b']}),
                                            table_name="#tt_null",
                                            force=True)
print(type(df))

**2. Invoke table function**

This function returns a dataframe which represents a specified table in SAP HANA

In [ ]:
df1 = connection_context.table("#tt_null")
print(type(df1))

## Property of dataframe

**SQL select statement**

In [ ]:
print(df.select_statement)

**Connection**

In [ ]:
df.connection_context

## Fetch data in SAP HANA to client
Fetch the first 5 rows of df into client as a <b>Pandas Dataframe</b>

In [ ]:
pd_df = df.head(5).collect()
print(pd_df)
print(type(pd_df))

## Save a dataframe

In [ ]:
# Creates a table or view holding the current DataFrame's data.
df.save(where="#TT")
print(connection_context.table("#TT").collect())

## Simple Operations

In this section, we will show some basic operations of hana-ml dataframe. Please refer more functions in detail in the dataframe documentation.

In hana-ml, we provide a class called DataSets which contains several small public datasets. You could use load_bank_data() to load the bank dataset. We will use the bank dataset in the following examples.

In [ ]:
# load the dataset and obtain a series of dataframe

sql_cmd = 'SELECT * FROM "BANK"'
bank_df = hdf.DataFrame(conn, sql_cmd)

print(bank_df.head(3).collect())
print(type(bank_df))

**columns**

In [ ]:
bank_df.columns

**shape**

In [ ]:
bank_df.shape

**get_table_structure**

In [ ]:
bank_df.get_table_structure()

**Add ID**

In [ ]:
bank_df.add_id(id_col='ID1').head(5).collect()

**Add a constant column**

In [ ]:
bank_df.add_constant(column_name='Constant', value=888).head(5).collect()

**Count the number of rows**

In [ ]:
bank_df.count()

**Drop duplicates**

In [ ]:
df_no_duplicate = bank_df.drop_duplicates()
print(df_no_duplicate.count())# there is no duplicate row in bank dataset
print(df_no_duplicate.select_statement)

#### Remove a column

In [ ]:
df1 = bank_df
print(df1.shape)
df2 = df1.drop(["LABEL"])
print(df2.shape)
print(df2.select_statement)

**Filtering Data**

In [ ]:
print(bank_df.filter('AGE > 60').head(3).collect())
print(bank_df.filter('AGE > 60').select_statement)

**Sorting**

In [ ]:
print(bank_df.filter('AGE>60').sort(['AGE']).head(3).collect())
print(bank_df.filter('AGE>60').sort(['AGE']).select_statement)

**Cast**

In [ ]:
bank_df.cast({"AGE": "BIGINT", "JOB": "NVARCHAR(50)"}).get_table_structure()

**Distinct**

In [ ]:
bank_df.distinct(cols='JOB').collect()

**Describing a dataframe**

In [ ]:
bank_df.describe().collect()

**Replace NULL value with a specified value**

In [ ]:
print(bank_df.collect())
print(bank_df.fillna(value=0).collect())

In [ ]:
print(bank_df.collect())
print(bank_df.fillna(value='').collect())

In [ ]:
print(df.collect())
print(df.fillna('').fillna(0).collect())

**Projection**

In [ ]:
dsp = bank_df.select("AGE", "JOB", ('"AGE"*2', "TWICE_AGE"))
print(dsp.head(5).collect())
print(dsp.select_statement)

**Simple Joins**

In [ ]:
df1 = dataframe.create_dataframe_from_pandas(connection_context=connection_context,
                                             pandas_df=pd.DataFrame({"ID": [1,2,3],
                                                                     "ID2": [1,2,3],
                                                                     "V1": [2,3,4]}),
                                             table_name="#tt1",
                                             force=True)
df2 = dataframe.create_dataframe_from_pandas(connection_context=connection_context,
                                             pandas_df=pd.DataFrame({"ID": [1,2],
                                                                     "ID2": [1,2],
                                                                     "V2": [2,3]}),
                                             table_name="#tt2",
                                             force=True)
df3 = dataframe.create_dataframe_from_pandas(connection_context=connection_context,
                                             pandas_df=pd.DataFrame({"ID": [1,2,5],
                                                                     "ID2": [1,2,5],
                                                                     "V3": [2,3,4],
                                                                     "V4": [3,3,3],
                                                                     "V5": ['a','a','b']}),
                                             table_name="#tt3",
                                             force=True)


In [ ]:
print(df1.collect())
print(df2.collect())
print(df3.collect())

In [ ]:
dfs = [df1.set_index("ID"), df2.set_index("ID"), df3.set_index("ID")]
print(dfs[0].join(dfs[1:]).collect())

In [ ]:
dfs = [df1.set_index(["ID", "ID2"]), df2.set_index(["ID", "ID2"]), df3.set_index(["ID", "ID2"])]
print(dfs[0].join(dfs[1:]).collect())

In [ ]:
print(dfs[0].union([dfs[0], dfs[0]]).collect())

**Sort by index**

In [ ]:
df1.sort_index().collect()

**Take min, max, sum, median, mean**

In [ ]:
df1.min()

In [ ]:
df1.select("V1").min()

In [ ]:
df1.max()

In [ ]:
df1.sum()

In [ ]:
df1.median()

In [ ]:
df1.mean()

**Value counts**

In [ ]:
df3.value_counts().collect()

**Split column**

In [ ]:
import pandas as pd
split_df = \
dataframe.create_dataframe_from_pandas(connection_context,
                                       pandas_df=pd.DataFrame({"ID": [1,2],
                                                               "COL": ['1,2,3', '3,4,4']}),
                                       table_name="#split_test",
                                       force=True)

In [ ]:
new_df = split_df.split_column(column="COL", separator=",", new_column_names=["COL1", "COL2", "COL3"])
new_df.collect()

**Concat columns**

In [ ]:
new_df.concat_columns(columns=["COL1", "COL2", "COL3"], separator=",").collect()

## Close the connection

In [ ]:
connection_context.close()

## Thank you!